In [102]:
import essentia.standard
from dtw import *
import os
import pretty_midi
from time import time
import tslearn.metrics

In [103]:
def writeMidi(pitch_sequence, onset, duration, filename):
    new_mid = pretty_midi.PrettyMIDI()
    new_ch = pretty_midi.Instrument(0)
    for x in range(0, len(pitch_sequence)):
        note = pretty_midi.Note(pitch=int(pitch_sequence[x]), velocity=100, start=onset[x], end=onset[x]+duration[x])
        new_ch.notes.append(note)
    new_mid.instruments.append(new_ch)
    new_mid.write('melodies_midi/' + filename + '.mid')

In [104]:
def calculate_gradient_vector(melody):
    _gradients = []

    for index, note in enumerate(melody):
        if index == 0:
            continue

        _gradients.append(note - melody[index - 1])

    return _gradients

In [105]:
loader = essentia.standard.EqloudLoader(filename='hummings/counting_stars.wav', sampleRate=44100)
audio = loader()
pitch_extractor = essentia.standard.PredominantPitchMelodia(frameSize=2048, hopSize=128)
pitch_values, pitch_confidence = pitch_extractor(audio)
contour_extractor = essentia.standard.PitchContourSegmentation()
onset, duration, MIDI_pitch = contour_extractor(pitch_values, audio)
gradient_query = calculate_gradient_vector(MIDI_pitch)

In [106]:
melodies = os.listdir('vocals/')

In [107]:
for cancion in melodies:
    loader_db = essentia.standard.EqloudLoader(filename="vocals/" + cancion, sampleRate=44100)
    audio_db = loader_db()
    pitch_extractor_db = essentia.standard.PredominantPitchMelodia()
    pitch_values_db, pitch_confidence_db = pitch_extractor_db(audio_db)
    #print(type(pitch_values_db))
    contour_extractor_db = essentia.standard.PitchContourSegmentation()
    onset_db, duration_db, MIDI_pitch_db = contour_extractor_db(pitch_values_db, audio_db)
    gradient_db = calculate_gradient_vector(MIDI_pitch_db)
    #writeMidi(MIDI_pitch_db, onset_db, duration_db, cancion)
    start_time=time()
    #alignment = dtw(pitch_values[pitch_values != 0], pitch_values_db[pitch_values_db != 0], step_pattern=rabinerJuangStepPattern(6, "c"), keep_internals=True, open_begin=True, open_end=True)
    #alignment = dtw(MIDI_pitch, MIDI_pitch_db, step_pattern=rabinerJuangStepPattern(6, "c"), keep_internals=True, open_begin=True, open_end=True)
    #alignment = dtw(MIDI_pitch, MIDI_pitch_db, keep_internals=True, open_end=True)
    alignment = dtw(gradient_query, gradient_db, keep_internals=True, open_end=True)
    #path, distance = tslearn.metrics.dtw_subsequence_path(MIDI_pitch, MIDI_pitch_db)
    #path, distance = tslearn.metrics.dtw_subsequence_path(gradient_query, gradient_db)
    #path, distance = tslearn.metrics.dtw_subsequence_path(pitch_values[pitch_values != 0], pitch_values_db[pitch_values_db != 0])
    elapsed_time=time()-start_time
    print(cancion)
    print(elapsed_time)
    print(alignment.distance)

all_star.wav
0.002477407455444336
256.0
cant_stop.wav
0.0027222633361816406
235.0
chumbawamba.wav
0.0025517940521240234
321.0
counting_stars.wav
0.0030977725982666016
232.0
radioactive.wav
0.002036571502685547
228.0
